In [ ]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt

from src.utils import print_yaml, bootstrap_sampling, interpolate_unique, rescale_logits, calculate_tau
from src.save_load import loadAudit, savePlot, loadFbdStudy, loadTargetSignals, loadShadowModelSignals
from LeakPro.leakpro.attacks.mia_attacks.lira import lira_vectorized
from LeakPro.leakpro.attacks.mia_attacks.rmia import rmia_vectorised, rmia_get_gtlprobs

In [ ]:
# -------------------------- #
#   Load fbd study results   #
# -------------------------- #
study_name = "cifar10-resnet-fbd-815409b641"
study_path = os.path.join("study", study_name)
metadata = []
gtl_probs = []
resc_logits = []

metadata, fbd_trial_results, gtl_probs, resc_logits, labels = loadFbdStudy(study_name, metadata=True, gtl=True, logits=True)
target_folder = metadata["study"]["target_folder"]

In [ ]:
# -------------------------------- #
#   Load baseline target signals   #
# -------------------------------- #
target_folder = metadata["study"]["target_folder"]
baseline_target_logits, baseline_target_inmask, baseline_target_metadata = loadTargetSignals(target_folder)

# Rescale baseline target logits
baseline_resc_logits = rescale_logits(baseline_target_logits, labels)
print(f"Target resc_logits: {baseline_resc_logits[:10]}, shape: {baseline_resc_logits.shape}")

# Calculate the GTL Probabilities for the target logits
target_gtl_probs = rmia_get_gtlprobs(baseline_target_logits, labels)
print(f"Target gtl_probs: {target_gtl_probs[:10]}, shape: {target_gtl_probs.shape}")

In [ ]:
# ----------------------------- #
#   Load shadow model signals   #
# ----------------------------- #
shadow_logits, shadow_inmask = loadShadowModelSignals(target_folder)
# Rescale and calc gtl for shadow models

# Calculate the GTL Probabilities for shadow model logits
N, M, C = shadow_logits.shape
shadow_gtl_probs_list = []
rescaled_shadow_logits_list = []

for m in range(M):
    model_logits = shadow_logits[:, m, :]  # shape (N, C)
    shadow_gtl_probs_list.append(rmia_get_gtlprobs(model_logits, labels))
    rescaled_shadow_logits_list.append(rescale_logits(model_logits, labels))
    print(f"{len(shadow_gtl_probs_list)} shadow gtl probs calculated and {len(rescaled_shadow_logits_list)} rescaled logits calculated")

shadow_gtl_probs = np.stack(shadow_gtl_probs_list, axis=1)  # shape = (N, M)
rescaled_shadow_logits = np.stack(rescaled_shadow_logits_list, axis=1)
print(f"shadow gtl probs shape: {shadow_gtl_probs.shape}, shadow resc logits shape: {rescaled_shadow_logits.shape}")

In [ ]:
# ----------------------------- #
#   Audit the baseline target   #
# ----------------------------- #
# Audit the baseline target with LiRA
lira_scores = lira_vectorized(baseline_resc_logits, rescaled_shadow_logits, shadow_inmask, "carlini", online=True)
print(f"lira_scores first 10: {lira_scores[:10]}")
# Audit the baseline target with RMIA
rmia_scores = rmia_vectorised(target_gtl_probs, shadow_gtl_probs, shadow_inmask, online=True, use_gpu_if_available=True)
print(f"rmia_scores first 10: {rmia_scores[:10]}")

# ----------------------------- #
#   Audit the weighted target   #
# ----------------------------- #
# Audit all weighted models with RMIA
weighted_rmia_scores = []
for gtl in gtl_probs:
    weighted_rmia_scores.append(rmia_vectorised(gtl, shadow_gtl_probs, shadow_inmask, online=True, use_gpu_if_available=True))
    print(f"Weighted rmia scores calculated: {len(weighted_rmia_scores)}")
    
# Audit all weighted models with RMIA
weighted_lira_scores = []
for resc_logits in resc_logits:
    weighted_lira_scores.append(lira_vectorized(resc_logits, rescaled_shadow_logits, shadow_inmask, "carlini", online=True))
    print(f"Weighted lira scores calculated: {len(weighted_lira_scores)}")
    
print(f"rmia_scores_weighted count: {len(weighted_rmia_scores)}")
print(f"lira_scores_weighted count: {len(weighted_lira_scores)}")

In [ ]:
# ----------------- #
#   Calculate tau   #
# ----------------- #
# Baseline
fpr1 = 0.1
fpr2 = 0.01
fpr3 = 0.001
tau_baseline = calculate_tau(rmia_scores, baseline_target_inmask, fpr1)
print(f"baseline tau: {tau_baseline} at fpr: {fpr1}")

# Weighted 0.1 fpr
weigted_taus_rmia_1 = []
weigted_taus_lira_1 = []
weigted_taus_rmia_2 = []
weigted_taus_lira_2 = []
weigted_taus_rmia_3 = []
weigted_taus_lira_3 = []

# --- RMIA ---
for w_rmia_score in weighted_rmia_scores:
    weigted_taus_rmia_1.append(calculate_tau(w_rmia_score, baseline_target_inmask, fpr1))
    weigted_taus_rmia_2.append(calculate_tau(w_rmia_score, baseline_target_inmask, fpr2))
    weigted_taus_rmia_3.append(calculate_tau(w_rmia_score, baseline_target_inmask, fpr3))
    print(f"Calculated rmia taus: {len(weigted_taus_rmia_3)}")
print(f"n rmia taus: {weigted_taus_rmia_1}, {weigted_taus_rmia_2}, {weigted_taus_rmia_3}")

# --- LIRA ---
for w_lira_score in weighted_lira_scores:
    weigted_taus_lira_1.append(calculate_tau(w_lira_score, baseline_target_inmask, fpr1))
    weigted_taus_lira_2.append(calculate_tau(w_lira_score, baseline_target_inmask, fpr2))
    weigted_taus_lira_3.append(calculate_tau(w_lira_score, baseline_target_inmask, fpr3))
    print(f"Calculated lira taus: {len(weigted_taus_lira_3)}")
print(f"n lira taus: {weigted_taus_lira_1}, {weigted_taus_lira_2}, {weigted_taus_lira_3}")

# Study outputs
accuracy = [res.accuracy for res in fbd_trial_results]
tau_rmia = [res.tau for res in fbd_trial_results]   # tau in this context is log(tauc_fbd@0.1/tauc_ref@0.1)
print(f"study tau: {tau_rmia}")

In [ ]:
# --------------------------------- #
#   Visualize Model Study Results   #
# --------------------------------- #


# Plot
fig, axes = plt.subplots(2, 2, figsize=(12, 5))
# --- Top-left: your study tau (FbD) --- #
ax = axes[0,0]
ax.scatter(tau_rmia, accuracy, label="FbD tau", color="purple")
ax.set_xlabel("tau@0.1 (log(tauc_fbd@0.1 / tauc_ref@0.1))")
ax.set_ylabel("Accuracy")
ax.set_title("FbD study")
ax.grid(True, alpha=0.3)

# --- Top-right: tau@0.1 for RMIA + LIRA --- #
ax = axes[0,1]
ax.scatter(weigted_taus_rmia_1, accuracy, label="RMIA", marker='o')
ax.scatter(weigted_taus_lira_1, accuracy, label="LIRA", marker='x')
ax.set_xlabel("τ@0.1")
ax.set_ylabel("Accuracy")
ax.set_title("τ at FPR = 0.1")
ax.grid(True, alpha=0.3)
ax.legend()

# --- Bottom-left: tau@0.01 --- #
ax = axes[1,0]
ax.scatter(weigted_taus_rmia_2, accuracy, label="RMIA", marker='o')
ax.scatter(weigted_taus_lira_2, accuracy, label="LIRA", marker='x')
ax.set_xlabel("τ@0.01")
ax.set_ylabel("Accuracy")
ax.set_title("τ at FPR = 0.01")
ax.grid(True, alpha=0.3)
ax.legend()

# --- Bottom-right: tau@0.001 --- #
ax = axes[1,1]
ax.scatter(weigted_taus_rmia_3, accuracy, label="RMIA", marker='o')
ax.scatter(weigted_taus_lira_3, accuracy, label="LIRA", marker='x')
ax.set_xlabel("τ@0.001")
ax.set_ylabel("Accuracy")
ax.set_title("τ at FPR = 0.001")
ax.grid(True, alpha=0.3)
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# -------------------------- #
#   Visualize LiRA results   #
# -------------------------- #